## Make and view auditory spectrograms

In [1]:
from pathlib import Path
from audspec import Audspec
from phonlab.utils import dir2df

## Customize

In [2]:
# Where to find the .wav files and textgrids.
wavdir = Path(
    'G:/.shortcut-targets-by-id/1HR3dpND6KDuywK8enT1fkmh7Fq1daZlC/09_25_22'
)
# Where to stash the spectrograms.
specdir = Path(
    'G:/.shortcut-targets-by-id/1HR3dpND6KDuywK8enT1fkmh7Fq1daZlC/09_25_22/audspec.new'
)

fs = 22050     # The .wav files will be resampled to the rate specified by `fs`.
step = 0.005  # The frame step for the spectrogram, in seconds.
maxpatterson_coef = 100
rfftparams = {            # Params passed to rfft() in make_zgram()
    'overwrite_x': True,  # Try to reduce memory usage
    'workers': -1         # Use all CPUs
}

## Create the objects used in the analysis

In [3]:
# The Audspec to use for each of the .wav files.
aud = Audspec(fs, step_size=step, maxcbfiltn=maxpatterson_coef)

# Create image filters.
sharp_1 = aud.create_sharp_filter(span=3, mult=2)
sharp_2 = aud.create_sharp_filter(span=6, mult=1)
temporal_sharp = aud.create_sharp_filter(span=0.05, mult=1, dimension="time")
blur = aud.create_blur_filter(span=3, sigma=3)

## Find files to analyze

In [4]:
wavdf = dir2df(wavdir, fnpat='\.wav$', addcols=['barename'])
print(f'Found {len(wavdf)} .wav files.')
wavdf

Found 6 .wav files.


,relpath,fname,barename
0,.,AN_F6_DI.wav,AN_F6_DI
1,.,AN_M2_CS.wav,AN_M2_CS
2,.,BA_F1_GL.wav,BA_F1_GL
3,.,BA_M2_LJ.wav,BA_M2_LJ
4,.,SA_F1_BP.wav,SA_F1_BP
5,.,SA_M3_CM.wav,SA_M3_CM


## Find cached spectrogram files

In [5]:
specdf = dir2df(specdir, fnpat='(?P<wavname>.*)\.audspec\.npz$')
print(f'Found {len(specdf)} .npz files.')
specdf

Found 3 .npz files.


C:\ProgramData\Miniconda3\envs\audspec\lib\site-packages\phonlab\utils.py:299: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, col] = df.loc[:, col].astype('category')


,relpath,fname,wavname
0,.,AN_F6_DI.audspec.npz,AN_F6_DI
1,.,AN_M2_CS.audspec.npz,AN_M2_CS
2,.,BA_F1_GL.audspec.npz,BA_F1_GL


## Merge to find .wav files that don't have a corresponding audspec file

In [6]:
todo = wavdf.merge(specdf, how='left', left_on='barename', right_on='wavname', suffixes=['_wav', '_spec'])
todo = todo[todo['fname_spec'].isna()]
print(f'Found {len(todo)} .wav files that need audspec processing.')
todo

Found 3 .wav files that need audspec processing.


,relpath_wav,fname_wav,barename,relpath_spec,fname_spec,wavname
3,.,BA_M2_LJ.wav,BA_M2_LJ,NaN,NaN,NaN
4,.,SA_F1_BP.wav,SA_F1_BP,NaN,NaN,NaN
5,.,SA_M3_CM.wav,SA_M3_CM,NaN,NaN,NaN


## Create and cache spectrograms

Create the auditory spectrograms and save in a `.npz` file. The `aud` object contains the acoustic and auditory spectrograms already, and you don't need to do anything special to save them when you call `savez()`. You can create other variables to be saved and pass them as additional parameters to `savez()`.

In [7]:
for row in todo.itertuples():
    aud.make_zgram(
        wavdir / row.relpath_wav / row.fname_wav,
        rfftparams
    )

    # Apply image filters to spectrogram.
    lgram = aud.apply_filt(aud.zgram, sharp_1, axis=0, half_rectify=True)
    igram = aud.apply_filt(lgram, blur, axis=0, half_rectify=False)
    lgram = aud.apply_filt(lgram, sharp_2, axis=0, half_rectify=True)
    ogram = aud.apply_filt(aud.zgram, temporal_sharp, axis=1, half_rectify=True)

    # Save to disk, adding filtered auditory spectrograms.
    specname = specdir / f'{row.barename}.audspec.npz'
    aud.savez(
        specname,
        igram=igram,   # Optional custom variables start from here.
        lgram=lgram,
        ogram=ogram,
        foo=1
    )
    print(f'Saved {specname}.')

Saved G:\.shortcut-targets-by-id\1HR3dpND6KDuywK8enT1fkmh7Fq1daZlC\09_25_22\audspec.new\BA_M2_LJ.audspec.npz.
Saved G:\.shortcut-targets-by-id\1HR3dpND6KDuywK8enT1fkmh7Fq1daZlC\09_25_22\audspec.new\SA_F1_BP.audspec.npz.
Saved G:\.shortcut-targets-by-id\1HR3dpND6KDuywK8enT1fkmh7Fq1daZlC\09_25_22\audspec.new\SA_M3_CM.audspec.npz.
